In [1]:
from __future__ import division
from keras.models import Model
from keras.layers import Conv2D,MaxPooling2D,AveragePooling2D
from keras.layers import Input,Activation,Dense,Flatten
from keras.layers.merge import add
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras import backend as K
from keras.utils import plot_model
import os
import six

Using TensorFlow backend.
c:\users\xiang\.conda\envs\fianl_even\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\xiang\.conda\envs\fianl_even\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\xiang\.conda\envs\fianl_even\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\xiang\.conda\envs\fianl_even\lib

In [2]:
def _handle_dim_ordering():
    global Row_axis
    global Col_axis
    global Channel_axis
    if K.image_data_format()=='tf':
        Row_axis=1
        Col_axis=2
        Channel_axis=3
    else:
        Channel_axis=1
        Row_axis=2
        Col_axis=3

def _get_block(identifier):
    if isinstance(identifier, six.string_types):
        res=globals().get(identifier)
        if not res:
            raise ValueError('Invalid{}'.format(identifier))
        return res
    return identifier


def _bn_relu(input):
    norm=BatchNormalization(axis=-1)(input)
    return Activation('relu')(norm)


def _conv_bn_relu(**conv_params):
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    def f(input):
        conv = Conv2D(filters=filters, kernel_size=kernel_size,strides=strides, padding=padding,kernel_initializer=kernel_initializer,kernel_regularizer=kernel_regularizer)(input)
        return _bn_relu(conv)
    return f

def _bn_relu_conv(**conv_params):
    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    def f(input):
        activation = _bn_relu(input)
        return Conv2D(filters=filters, kernel_size=kernel_size,strides=strides, padding=padding, kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer)(activation)
    return f

def _shortcut(input,residual):
    
    input_shape=K.int_shape(input)
    residual_shape=K.int_shape(residual)
    stride_width=int(round(input_shape[1]/residual_shape[1]))
    stride_height=int(round(input_shape[2]/residual_shape[2]))
    equal_channels=input_shape[3]==residual_shape[3]
    shortcut=input
    if stride_width > 1 or stride_height > 1 or not equal_channels:
        shortcut = Conv2D(filters=residual_shape[3], kernel_size=(1, 1), strides=(stride_width, stride_height), padding="valid", kernel_initializer="he_normal",kernel_regularizer=l2(0.0001))(input)
    return add([shortcut, residual])

def _residual_block(block_function, filters, repetitions, is_first_layer=False):
    def f(input):
        for i in range(repetitions):
            init_strides = (1, 1)
            if i == 0 and not is_first_layer:
                init_strides = (2, 2)
            input = block_function(filters=filters, init_strides=init_strides, is_first_block_of_first_layer=(is_first_layer and i == 0))(input)
        return input
    return f

def basic_block(filters, init_strides=(1, 1), is_first_block_of_first_layer=False):
    def f(input):
        if is_first_block_of_first_layer:
            # don't repeat bn->relu since we just did bn->relu->maxpool
            conv1 = Conv2D(filters=filters, kernel_size=(3, 3),strides=init_strides, padding="same", kernel_initializer="he_normal", kernel_regularizer=l2(1e-4))(input)
        else:
            conv1 = _bn_relu_conv(filters=filters, kernel_size=(3, 3), strides=init_strides)(input)
        residual = _bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv1)
        return _shortcut(input, residual)
    return f

def bottleneck(filters, init_strides=(1, 1), is_first_block_of_first_layer=False):
    def f(input):
        if is_first_block_of_first_layer:
            # don't repeat bn->relu since we just did bn->relu->maxpool
            conv_1_1 = Conv2D(filters=filters, kernel_size=(1, 1), strides=init_strides, padding="same", kernel_initializer="he_normal", kernel_regularizer=l2(1e-4))(input)
        else:
            conv_1_1 = _bn_relu_conv(filters=filters, kernel_size=(1, 1), strides=init_strides)(input)

        conv_3_3 = _bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv_1_1)
        residual = _bn_relu_conv(filters=filters * 4, kernel_size=(1, 1))(conv_3_3)
        return _shortcut(input, residual)
    return f

In [3]:
class ResnetBuilder(object):
    @staticmethod
    def build(input_shape, num_outputs, block_fn, repetitions):
        block_fn = _get_block(block_fn)

        input = Input(shape=input_shape)
        conv1 = _conv_bn_relu(filters=64, kernel_size=(7, 7), strides=(2, 2))(input)
        pool1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(conv1)

        block = pool1
        filters = 64
        for i, r in enumerate(repetitions):
            block = _residual_block(block_fn, filters=filters, repetitions=r, is_first_layer=(i == 0))(block)
            filters *= 2

        # Last activation
        block = _bn_relu(block)

        # Classifier block
        block_shape = K.int_shape(block)
        pool2 = AveragePooling2D(pool_size=(block_shape[1], block_shape[2]), strides=(1,1))(block)
        flatten1 = Flatten()(pool2)
        dense = Dense(units=num_outputs, kernel_initializer="he_normal", activation="sigmoid")(flatten1)

        model = Model(inputs=input, outputs=dense)
        return model


    @staticmethod
    def build_resnet_18(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, basic_block, [2, 2, 2, 2])

    @staticmethod
    def build_resnet_34(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, basic_block, [3, 4, 6, 3])

    @staticmethod
    def build_resnet_50(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 4, 6, 3])

    @staticmethod
    def build_resnet_101(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 4, 23, 3])

    @staticmethod
    def build_resnet_152(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 8, 36, 3])


In [4]:
image_size=224
iterations=10000
img_rows, img_cols = 224,224
img_channels = 1
num_classes= 6
batch_size=25        
epochs=10
# iterations = 782
weight_decay= 1e-4
model=ResnetBuilder.build_resnet_50((img_rows,img_cols,img_channels),6)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 64) 3200        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 112, 112, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [5]:
import numpy as np
import pandas as pd
data_path="C:/Users/xiang/Desktop/rsna-intracranial-hemorrhage-detection-224"
images_path=f'{data_path}/stage_2_train_images'
csv_path=f'{data_path}/stage_2_train.csv'
image_index=os.listdir(f'{data_path}/stage_2_train_images')
image_index=pd.DataFrame(image_index)
image_index=image_index[0].str.slice(stop=12).values

In [6]:
def shuffle_data(x):
    rand_indices=np.random.permutation(x.shape[0])
    train_indices=rand_indices[0:650000]
    vali_indices=rand_indices[650000:755940]
    return train_indices,vali_indices

In [7]:
import augmentation as aug
gener=aug.DataGenerator(data_path,images_path=f'{data_path}/stage_2_train_images',csv_path=f'{data_path}/stage_2_train.csv',
                    num_thread=None,is_train=True)

tr_indices,vali_indices=shuffle_data(image_index)
print('train shape:',np.shape(tr_indices))
print('vali shape:',np.shape(vali_indices))

train shape: (650000,)
vali shape: (105940,)


In [8]:
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint

# sgd = optimizers.SGD(lr=.1, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['binary_accuracy'])


reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1,patience=10, min_lr=0.001)
checkpointer=ModelCheckpoint(filepath=os.path.join(data_path,'chack_point_weight.hdf5'),save_best_only=True,save_weights_only=True)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
model.load_weights(os.path.join(data_path,'my_model_weights.h5'))

In [10]:
history=model.fit_generator(generator=gener.get_X_batch(image_index[tr_indices],batch_size,image_size),steps_per_epoch=iterations,epochs=epochs,callbacks=None,
                            validation_data=gener.get_X_batch(image_index[vali_indices],20,image_size),validation_steps=int(len(vali_indices)//20),class_weight={0:2.,1:1.,2:1.,3:1.,4:1.,5:1.})
print(history)

Epoch 1/10
10000/10000 [==============================] - 10160s 1s/step - loss: 0.5235 - binary_accuracy: 0.9063 - val_loss: 0.2476 - val_binary_accuracy: 0.8839
Epoch 2/10
10000/10000 [==============================] - 10134s 1s/step - loss: 0.5230 - binary_accuracy: 0.9065 - val_loss: 0.4025 - val_binary_accuracy: 0.8824
Epoch 3/10
10000/10000 [==============================] - 10131s 1s/step - loss: 0.5208 - binary_accuracy: 0.9070 - val_loss: 0.4048 - val_binary_accuracy: 0.8833
Epoch 4/10
10000/10000 [==============================] - 10155s 1s/step - loss: 0.5208 - binary_accuracy: 0.9071 - val_loss: 0.4376 - val_binary_accuracy: 0.8424
Epoch 5/10
 9822/10000 [============================>.] - ETA: 2:44 - loss: 0.5170 - binary_accuracy: 0.9082

KeyboardInterrupt: 

In [11]:

test_image_path=f'{data_path}/stage_2_test_images'
test_image_index=os.listdir(f'{data_path}/stage_2_test_images')
test_image_index=pd.DataFrame(test_image_index)
test_image_index=test_image_index[0].str.slice(stop=12).values




In [12]:
from concurrent.futures import ThreadPoolExecutor as threadPool
from PIL import Image
def load_image(temp_path):
        im = np.array(Image.open(test_image_path+'/'+temp_path+'.png'))
#         print(im)
        return im
        
def get_img(img_paths, img_size):
        p = threadPool()
        X = np.zeros((len(img_paths), img_size, img_size), dtype=np.uint8)
        i = 0
        for future in p.map(load_image, img_paths):
            img = np.resize(future, (img_size, img_size))
#             print(img)
            X[i, :, :] = img
            
            i += 1
        p.shutdown(wait=False)
        return X

In [13]:
def get_test_batch(X_path,batch_size,img_size):
        try:
            if len(X_path) %batch_size != 0:
                raise Exception("batchSize not match the size of data!")
        except Exception as err:
            print(err)
        else:
            while 1:
                for i in range(0, len(X_path), batch_size):
                    X =(get_img(X_path[i:i + batch_size], img_size))
                    yield np.resize(X/255,(len(X),img_size,img_size,1)).astype('float16')

In [14]:
test_gener=get_test_batch(test_image_index,16,224)

In [15]:
submission_result=model.predict_generator(generator=test_gener,steps=len(test_image_index)//16)
print(submission_result)

[[7.09203541e-01 1.04035586e-01 6.78669214e-02 8.12038779e-03
  2.50184596e-01 3.65378052e-01]
 [3.44991684e-04 2.68220901e-07 7.18235970e-06 4.17232513e-07
  6.23762608e-05 1.58518553e-04]
 [7.83329904e-02 1.92481279e-03 9.18483734e-03 1.05735362e-02
  3.78098488e-02 2.21931636e-02]
 ...
 [1.86298370e-01 3.76932919e-02 4.34757769e-02 1.22469068e-02
  7.23642111e-02 8.86104405e-02]
 [1.45331979e-01 7.06779957e-03 1.00631416e-02 3.51577997e-04
  5.93131185e-02 5.82143664e-02]
 [2.20509678e-01 3.56218219e-02 3.83265316e-02 4.08571959e-03
  1.12340510e-01 1.58711851e-01]]


In [16]:
def evaluation(prob):        
    return max(min(prob,1-pow(10,-15)),pow(10,-15))

In [17]:
for i in range(len(submission_result)):
    for j in range(len(submission_result[i])):
        submission_result[i,j]=evaluation(submission_result[i,j])

In [18]:
def read_csv(filename):
            df = pd.read_csv(filename)
            df["ImageID"] = df["ID"].str.slice(stop=12)
            df["Diagnosis"] = df["ID"].str.slice(start=13)
            df = df.loc[:, ["Label", "Diagnosis", "ImageID"]]
            df = df.set_index(['ImageID', 'Diagnosis']).unstack(level=-1)
            return df

In [20]:
submission_csv=read_csv('C:/Users/xiang/Desktop/rsna-intracranial-hemorrhage-detection-224/stage_2_sample_submission.csv')
submission_csv.head()

Label                                                          \
Diagnosis      any epidural intraparenchymal intraventricular subarachnoid   
ImageID                                                                      
ID_000000e27   0.5      0.5              0.5              0.5          0.5   
ID_000009146   0.5      0.5              0.5              0.5          0.5   
ID_00007b8cb   0.5      0.5              0.5              0.5          0.5   
ID_000134952   0.5      0.5              0.5              0.5          0.5   
ID_000176f2a   0.5      0.5              0.5              0.5          0.5   

                       
Diagnosis    subdural  
ImageID                
ID_000000e27      0.5  
ID_000009146      0.5  
ID_00007b8cb      0.5  
ID_000134952      0.5  
ID_000176f2a      0.5

In [21]:
submission_csv.iloc[:, :] =submission_result

print(submission_csv.head())

                 Label                                                  \
Diagnosis          any      epidural intraparenchymal intraventricular   
ImageID                                                                  
ID_000000e27  0.709204  1.040356e-01         0.067867     8.120388e-03   
ID_000009146  0.000345  2.682209e-07         0.000007     4.172325e-07   
ID_00007b8cb  0.078333  1.924813e-03         0.009185     1.057354e-02   
ID_000134952  0.115158  1.035544e-02         0.023400     3.395021e-03   
ID_000176f2a  0.206377  9.338051e-03         0.042480     1.945603e-02   

                                     
Diagnosis    subarachnoid  subdural  
ImageID                              
ID_000000e27     0.250185  0.365378  
ID_000009146     0.000062  0.000159  
ID_00007b8cb     0.037810  0.022193  
ID_000134952     0.058935  0.054969  
ID_000176f2a     0.051086  0.114467  


In [22]:
submission_csv = submission_csv.stack().reset_index()
submission_csv.insert(loc=0, column='ID', value=submission_csv['ImageID'].astype(str) + "_" + submission_csv['Diagnosis'])
submission_csv= submission_csv.drop(["ImageID", "Diagnosis"], axis=1)
submission_csv.to_csv('stage_2_sample_Resnet2.csv', index=False)
submission_csv.head()

,ID,Label
0,ID_000000e27_any,0.709204
1,ID_000000e27_epidural,0.104036
2,ID_000000e27_intraparenchymal,0.067867
3,ID_000000e27_intraventricular,0.008120
4,ID_000000e27_subarachnoid,0.250185


In [23]:
model.save_weights(os.path.join(data_path,'my_model_weights.h5'))
model.save(os.path.join(data_path,'my_model.h5'))

In [ ]:
print(test_image_index)

In [ ]:
print(submission_result)

In [ ]:
result_dict={}
for i in range(len(test_image_index)):
    result_dict[test_image_index[i]]=submission_result[i]

In [ ]:
for i in result_dict.keys():
    print(i)

In [ ]:
submission_csv=read_csv('C:/Users/xiang/Desktop/rsna-intracranial-hemorrhage-detection-224/stage_2_sample_submission.csv')
submission_csv.head()

In [ ]:
for i in submission_csv.index:
    submission_csv.loc[i]=result_dict[i]

In [ ]:
submission_csv.head()

In [42]:
submission_csv = submission_csv.stack().reset_index()
submission_csv.insert(loc=0, column='ID', value=submission_csv['ImageID'].astype(str) + "_" + submission_csv['Diagnosis'])
submission_csv= submission_csv.drop(["ImageID", "Diagnosis"], axis=1)
submission_csv.to_csv('stage_2_sample_submission.csv', index=False)
submission_csv.head()

,ID,Label
0,ID_000000e27_any,0.666681
1,ID_000000e27_epidural,0.116979
2,ID_000000e27_intraparenchymal,0.143589
3,ID_000000e27_intraventricular,0.027768
4,ID_000000e27_subarachnoid,0.235570
